## Set UP:

In [1]:
!pip install h5py
!pip install openai
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 2.8 MB/s eta 0:00:0000:0100:01m


In [2]:
import ast
import h5py    
import transformers
import pandas as pd
from transformers import AutoTokenizer, pipeline
import csv
import torch

2024-05-13 12:59:03.068601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 12:59:03.068688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 12:59:03.162085: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Falcon --> for generate caption :

In [3]:
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    truncation=True,  
    pad_token_id=tokenizer.eos_token_id 
)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
'''filename="/kaggle/input/captionsforv0/generated_instruction-1(v0).csv"
with open(filename, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # Extract data from the CSV row
        description = row['Description']
        instructions = ast.literal_eval(row['Generated Instructions']) 
        image_index = row["image_index"]
        print(image_index)
        print("Original Description:")
        print(description)
        
        for idx, instruction in enumerate(instructions, start=1):
            print(f"\nInstruction {idx}:")
            print(instruction)
            sequences = pipeline(
                f"Given this original description:\n\n'{description}',\n\n update it according to the given instruction :\n\n'{instruction}'\n\nBut remain faithful to the structure of the original description and don't change it also keep faithful to the structure and the arrangement of each part in the original text.\n\nNew description:\n",
                max_length=2000,
                do_sample=True,
                top_k=10,
                temperature=0.1,
                top_p=0.9,
                no_repeat_ngram_size=3,
                repetition_penalty=2.0,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
            )
        
            generated_text = sequences[0]['generated_text'].strip()
            
            print("\nGenerated Description:")
            print(generated_text)
            print()'''

In [4]:
filename="/kaggle/input/dfcvbgfd/generated_instruction-1 (4).csv"
output_filename = "output_descriptions.csv"  

with open(filename, 'r', encoding='utf-8') as csvfile, open(output_filename, 'w', newline='', encoding='utf-8') as output_csvfile:
    reader = csv.DictReader(csvfile)
    fieldnames = ['image_index', 'instruction', 'original_description', 'new_description']
    writer = csv.DictWriter(output_csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in reader:
        image_index = row["image_index"]
        description = row['Description']
        instructions = ast.literal_eval(row['Generated Instructions'])
        
        for idx, instruction in enumerate(instructions, start=1):
            print(f"\nInstruction {idx}:")
            print(instruction)
            sequences = pipeline(
                f"Given this original description:\n\n'{description}',\n\n update the given description according to the given instruction :\n\n'{instruction}'\n\nBut remain faithful to the structure of the given description and don't change the structure of it, also keep faithful to the structure and the arrangement of each part in the original text but don't add any additional description.\n\nNew description:\n",
                max_length=2000,
                do_sample=True,
                top_k=10,
                temperature=0.1,
                top_p=0.9,
                no_repeat_ngram_size=3,
                repetition_penalty=2.0,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
            )
        
            generated_text = sequences[0]['generated_text'].strip()
            new_description_index = generated_text.find("New description:\n")
            print(generated_text)
            if new_description_index != -1:
                new_description = generated_text[new_description_index + len("New description:\n"):]
            else:
                new_description = generated_text
                
            writer.writerow({
                'image_index': image_index,
                'instruction': instruction,
                'original_description': description,
                'new_description': new_description
            })

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Instruction 1:
Change   the color of the garment to black
Given this original description:

'full pose of JEANS for Men. Skinny-fit jeans in blue. Fading, distressing, and paint splatter throughout in white and blue. Five-pocket styling. Leather logo patch at back pocket in orange. Contrast stitching golden yellow. Logo flag at button-fly.',

 update the given description according to the given instruction :

'Change   the color of the garment to black'

But remain faithful to the structure of the given description and don't change the structure of it, also keep faithful to the structure and the arrangement of each part in the original text but don't add any additional description.

New description:

Black skinny-fit JEANS with fading, distressing and paint spatter throughout. The five-pocket style has a leather logo patch on the back pocket, while contrast stitching is golden yellow and there is an orange flag at the button-up fly.

Instruction 2:
Change   the fabric of the garment f

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104

Given this original description:

'full pose of JEANS for Men. Skinny-fit jeans in blue. Fading, distressing, and paint splatter throughout in white and blue. Five-pocket styling. Leather logo patch at back pocket in orange. Contrast stitching golden yellow. Logo flag at button-fly.',

 update the given description according to the given instruction :

'Change   the fabric shirt to polyester.'

But remain faithful to the structure of the given description and don't change the structure of it, also keep faithful to the structure and the arrangement of each part in the original text but don't add any additional description.

New description:

The updated description is as follows:

<p>A full-length pair of men's polyester shirts in a range of colors, including black, navy, and red. The shirts feature a classic collar with a single button-down front, long sleeves with adjustable cuffs, and a straight hem. They are made from lightweight, breathable fabrics that provide comfort and style.</